In [205]:
## Instruction: 
## https://darksky.net/dev/docs
## https://darksky.net/dev/docs/libraries

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import time, calendar
from datetime import datetime, timedelta

In [7]:
# generate the list of days defined between start and end day
def calendar_test(day, count, date_list = None):
    if date_list is None:
        date_list = []
    if count == 0:
        return date_list
    count -= 1
    day = day + timedelta(days=1)
    date = day.isoformat().split('T')
    date_list.append(" ".join(date))
    
    calendar_test(day,count,date_list)
    
    return date_list

# convert utc time to epoch time
def convert_utc_to_epoch(timestamp_string):
    '''Use this function to convert utc to epoch'''
    timestamp = datetime.strptime(timestamp_string, '%Y-%m-%d %H:%M:%S')
    epoch = int(calendar.timegm(timestamp.utctimetuple()))

    return epoch

# convert epoch time to YYYYMMDDHHMM format (year-month-day-hour-minute)
def date_time_convert(item):
    
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(item)).split(' ')
    date = date_time[0].split('-')
    time_ = date_time[1].split(':')
    date = 10000*int(date[0])+100*int(date[1])+1*int(date[2])
    time_ = 100*int(time_[0])+1*int(time_[1])
    date_time = date*10000+time_
    
    return date_time


def darksky_api_call(start_day,end_day):
    start_day = datetime.strptime(start_day, '%Y-%m-%d')
    end_day = datetime.strptime(end_day, '%Y-%m-%d')
    total_day = abs((end_day - start_day).days)

    day_list = calendar_test(start_day,total_day)

    # darksky website
    darksky = "https://api.darksky.net/forecast/"
    # API key
    key = "69fcefff98d6c6a0f9b0ebfec1e6f2cc"
    # Position: Amherst
    lon_lat = "/42.3601, -71.0589,"
    # Offset
    offset = "?exclude=minutely,flags"

    # define parameter you want to collect
    info_list = ['time','icon','temperature','humidity','pressure','cloudCover','windSpeed','uvIndex','visibility']
    info = {para:[] for para in info_list}

    # API calling
    for day in day_list:
    
        epoch_time = convert_utc_to_epoch(day)    
        link = darksky + key + lon_lat + str(epoch_time) + offset

        res = requests.get(link)
        data = res.json()
    
        if 'hourly' not in data:
            continue
        elif 'data' not in data['hourly']:
            continue
    
        for item in data['hourly']['data']:
            epoch = item['time']
            date_time = date_time_convert(item['time'])
            # parameters collected
            for para in info_list:
                if para not in item:
                    info[para].append(float('nan'))
                else:
                    info[para].append(item[para])
        
    file = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in info.items() ]))
    
    return file

In [10]:
# define start and end day
start_day = '2019-1-28'
end_day = '2019-1-29'

file = darksky_api_call(start_day,end_day)

# comment out if you want to save the data
# export_csv = file.to_csv ("file_name.csv", index = True, header=True)

In [12]:
# run this code if you want to convert epoch time to YYYYMMDDHHMM format (year-month-day-hour-minute)
date = []
for i in file['time']:
    date.append(date_time_convert(i))
    
file['date_time'] = date

# comment out if you want to save the data
# export_csv = file.to_csv ("file_name.csv", index = True, header=True)

In [14]:
file_list = ['darksky_data_2017_9_11_2019_1_27.csv','test.csv']
df = pd.DataFrame()
for i in file_list:
    df_file = pd.read_csv(i, index_col=0)
    df = df.append(df_file)
# df = df.set_index('date_time')
# export_csv = df.to_csv ("new_test.csv", index = True, header=True)

In [19]:
df = pd.read_csv('test.csv', index_col=0)
df = df.set_index('date_time')

file = pd.read_csv('darksky_data_2017_9_11_2019_1_27.csv', index_col=0)
df_new = pd.DataFrame()
df_new = df_new.append(file)
df_new = df_new.append(df)

In [23]:
export_csv = df_new.to_csv ("darksky_weather_data.csv", index = True, header=True)

In [32]:
weather_data = {}
weather_file = pd.read_csv('darksky_weather_data.csv', index_col=0)
weather_file['index'] = [i for i in range(len(weather_file))]
weather_file = weather_file.set_index('index')

for i in range(len(weather_file)):
    weather_data[str(weather_file.loc[i]['time'])[:8]]=weather_file.loc[i]['icon']

# days = []
# weathers = []
# for day, weather in weather_data.items():
#     days.append(day)
#     weathers.append(weather)
    
# for i, weather in enumerate(weathers):
#     if weather == 'clear-night':
#         weathers[i] = 'sunny'
#     elif weather == 'partly-cloudy-night':
#         weathers[i] = 'cloudy'

In [37]:
weather_file['date_time'] = [date_time_convert(item) for item in weather_file['time']]

In [39]:
for i in range(len(weather_file)):
    weather_data[str(weather_file.loc[i]['date_time'])[:8]]=weather_file.loc[i]['icon']

In [41]:
days = []
weathers = []
for day, weather in weather_data.items():
    days.append(day)
    weathers.append(weather)
    
for i, weather in enumerate(weathers):
    if weather == 'clear-night':
        weathers[i] = 'sunny'
    elif weather == 'partly-cloudy-night':
        weathers[i] = 'cloudy'

In [48]:
export_csv = weather_file.to_csv ("darksky_weather_data.csv", index = True, header=True)